
# Thermal Noise

This example demonstrates how thermal noise influences the raw signal in a flow cytometer detector.
We simulate thermal noise for detectors at two different temperatures and compare the resulting
signal characteristics.

Thermal noise, also known as Johnson-Nyquist noise, is generated by the thermal agitation of charge
carriers in the detector's resistive components. Its magnitude is proportional to the temperature
of the detector.


In [ ]:
from FlowCyPy.detector import Detector
from FlowCyPy import units
import matplotlib.pyplot as plt
from FlowCyPy.signal_digitizer import SignalDigitizer
from FlowCyPy import NoiseSetting

# Configure Noise Settings
NoiseSetting.include_noises = True
NoiseSetting.include_shot_noise = False
NoiseSetting.include_dark_current_noise = False
NoiseSetting.include_thermal_noise = True
NoiseSetting.include_RIN_noise = False

# Define simulation parameters
temperatures = [50, 500]  # Temperatures in Kelvin

signal_digitizer = SignalDigitizer(
    bit_depth='14bit',
    saturation_levels='auto',
    sampling_freq=1e6 * units.hertz,
)

# Initialize the plot
figure, (ax_signal, ax1) = plt.subplots(2, 1, figsize=(10, 4))

# Simulate thermal noise for different temperatures
for temperature in temperatures:
    # Initialize Detector
    detector = Detector(
        name=str(temperature),
        responsitivity=1 * units.ampere / units.watt,
        resistance=50 * units.ohm,
        numerical_aperture=0.2 * units.AU,
        phi_angle=0 * units.degree,
        temperature=temperature * units.kelvin,
    )
    dataframe = detector.get_initialized_signal(run_time=200e-6 * units.second, signal_digitizer=signal_digitizer)

    # Add thermal noise to the raw signal
    detector._add_thermal_noise_to_raw_signal(dataframe['Signal'])

    detector.capture_signal(dataframe['Signal'])

    # Plot the raw signal on the first axis
    ax_signal.step(
        dataframe.index,
        dataframe.Signal.pint.quantity.magnitude
    )

    # Plot the histogram of the raw signal on the bottom subplot
    ax1.hist(dataframe['Signal'], alpha=0.5, label=f"{detector.name} K", bins=50)

# Add legends and show the plot
ax1.legend(title="Temperature")
ax_signal.set_title("Raw Signal with Thermal Noise")
ax1.set_title("Histogram of Raw Signal")
plt.tight_layout()
plt.show()